In [1]:
#import libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
import keras
from keras.models import Sequential
from keras.layers import Dense
warnings.filterwarnings('ignore')
get_ipython().magic(u'matplotlib inline')

C:\Users\kumar.vishwesh\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#read healthy and non healthy dataset
df_healthy = pd.read_csv('Healthy_Data.csv')
df_unhealthy=pd.read_csv('Not_Healthy_Data.csv')

In [3]:
#adding label- 1 denotes healthy status
df_healthy['LABEL']=1
#adding label- 0 denotes unhealthy status
df_unhealthy['LABEL']=0
print(df_healthy.shape)
print(df_unhealthy.shape)

(798, 55)
(693, 55)


In [4]:
#down sampling for class balance
df_healthy = df_healthy.sample(n=693)

In [5]:
#merging
df = df_healthy.append(df_unhealthy, ignore_index=True)

In [6]:
# list of columns with only one unique values including NAN and dropping them from df
l=[]
for c in df.columns:
    if df[c].nunique(dropna=False)==1 :
        l.append(c)
df.drop(l, axis=1, inplace=True)

In [7]:
df.shape

(1386, 35)

In [8]:
#in merged and not in healthy
l1=list(set(df.columns.tolist())-set(df_healthy.columns.tolist()))
print(l1)
#in merged and not in unhealthy
l2=list(set(df.columns.tolist())-set(df_unhealthy.columns.tolist()))
print(l2)
#dropping uncommon
l=l1+l2
df.drop(l, axis=1, inplace=True)
df.shape

[]
[]


(1386, 35)

In [9]:
df.drop_duplicates()
df.shape

(1386, 35)

In [10]:
df.drop(['_raw','_time','punct','timestamp'], axis=1, inplace=True)
df.shape

(1386, 31)

In [11]:
l=[]
for col in df.columns:
    if ('ip' in col.lower()):
        l.append(col)
print(l)

['description']


In [12]:
l=[]
for col in df.columns:
    if ('date' in col.lower()):
        l.append(col)
print(l)

['date_hour', 'date_mday', 'date_minute', 'date_month', 'date_second', 'date_wday', 'date_year', 'date_zone']


In [13]:
df.drop(l, axis=1, inplace=True)
df.shape

(1386, 23)

In [14]:
l=[]
for col in df.columns:
    if ('id' in col.lower()):
        l.append(col)
        print(col)
        print(df[col].nunique())
print(l)

failed_test_id
88
id
828
['failed_test_id', 'id']


In [15]:
#df of non numeric columns
obj_df = df.select_dtypes(include=['object']).copy()

In [16]:
obj_col=obj_df.columns.tolist()
num_col=list(set(df.columns.tolist())-set(obj_col))
num_col.remove('LABEL')
num_df=df[num_col]
label_df=df[['LABEL']]
print(obj_df.shape)
print(num_df.shape)
print(label_df.shape)

(1386, 19)
(1386, 3)
(1386, 1)


In [17]:
obj_df.columns

Index(['action', 'channel', 'description', 'element', 'failed_test_action',
       'failed_test_id', 'failure_message', 'host', 'id', 'method',
       'result_code', 'source', 'sourcetype', 'url.host{}', 'url.path{}',
       'url.protocol', 'url.query{}.key', 'url.query{}.value',
       'where_isfailed'],
      dtype='object')

In [18]:
num_df.columns

Index(['exec_time', 'timestartpos', 'timeendpos'], dtype='object')

In [19]:
num_df.nunique()

exec_time       409
timestartpos      1
timeendpos        1
dtype: int64

In [20]:
obj_df.nunique()

action                 21
channel                 4
description             1
element                 1
failed_test_action      9
failed_test_id         88
failure_message        73
host                    2
id                    828
method                  2
result_code             4
source                  5
sourcetype              4
url.host{}              6
url.path{}             21
url.protocol            2
url.query{}.key         6
url.query{}.value       6
where_isfailed          3
dtype: int64

In [21]:
#missing values
na_num_col_list=num_df.columns[num_df.isna().any()].tolist()
na_num_col_list

['exec_time', 'timestartpos', 'timeendpos']

In [22]:
#imputing numerical columns
from sklearn.preprocessing import Imputer
imputer = Imputer(missing_values = 'NaN', strategy = 'mean', axis = 0)
imputer = imputer.fit(num_df[na_num_col_list].values)
X=imputer.transform(num_df[na_num_col_list].values)

In [23]:
df2 = pd.DataFrame(X,columns=na_num_col_list)
num_df.drop(na_num_col_list, axis=1, inplace=True)
num_df = pd.concat([num_df,df2],axis=1)
num_df.isna().any()

exec_time       False
timestartpos    False
timeendpos      False
dtype: bool

In [24]:
num_df.head()

,exec_time,timestartpos,timeendpos
0,1943.574879,0.0,23.0
1,14.000000,0.0,23.0
2,1943.574879,0.0,23.0
3,293.000000,0.0,23.0
4,1943.574879,0.0,23.0


In [25]:
obj_label_df=pd.concat([obj_df,label_df],axis=1)

In [26]:
class ChiSquare:
    def __init__(self, dataframe):
        self.df = dataframe
        self.p = None #P-Value
        self.chi2 = None #Chi Test Statistic
        self.dof = None
        self.dfTabular = None
        self.dfExpected = None
    def _print_chisquare_result(self, colX, alpha):
        #result = ""
        if self.p<alpha:
            #result="{0} is IMPORTANT for Prediction".format(colX)
            #print(result)
            return 0
        else:
            #result="{0} is NOT an important predictor. (Discard {0} from model)".format(colX)
            #print(result)
            return 1
            
        
    def TestIndependence(self,colX,colY, alpha=0.05):
        X = self.df[colX].astype(str)
        Y = self.df[colY].astype(str)
        
        self.dfObserved = pd.crosstab(Y,X) 
        chi2, p, dof, expected = stats.chi2_contingency(self.dfObserved.values)
        self.p = p
        self.chi2 = chi2
        self.dof = dof 
        
        self.dfExpected = pd.DataFrame(expected, columns=self.dfObserved.columns, index = self.dfObserved.index)
        
        output=self._print_chisquare_result(colX, alpha)
        return output

In [27]:
#Initialize ChiSquare Class
cT = ChiSquare(obj_label_df)

In [28]:
l_exclude=[]
for var in obj_df.columns.tolist():
    if cT.TestIndependence(colX=var,colY="LABEL" )==1:
        l_exclude.append(var)
l_exclude

['action',
 'description',
 'element',
 'failed_test_id',
 'failure_message',
 'host',
 'id',
 'method',
 'url.path{}',
 'url.protocol']

In [29]:
obj_df.drop(l_exclude, axis=1, inplace=True)
obj_df.shape

(1386, 9)

In [30]:
obj_df.nunique()

channel               4
failed_test_action    9
result_code           4
source                5
sourcetype            4
url.host{}            6
url.query{}.key       6
url.query{}.value     6
where_isfailed        3
dtype: int64

In [31]:
print(obj_df.shape)
print(num_df.shape)
print(label_df.shape)

(1386, 9)
(1386, 3)
(1386, 1)


In [32]:
#one hot encoding of categorical columns
obj_col=obj_df.columns.tolist()
obj_df=pd.get_dummies(obj_df, columns=obj_col, prefix=obj_col)

In [33]:
obj_df.shape

(1386, 47)

In [34]:
#Initialize ChiSquare Class
cT2 = ChiSquare(pd.concat([obj_df, label_df],axis=1))

In [ ]:
#list
l_exclude=[]
for var in obj_df.columns.tolist():
    if cT2.TestIndependence(colX=var,colY="LABEL" )==1:
        l_exclude.append(var)
l_exclude

In [36]:
obj_df.drop(l_exclude, axis=1, inplace=True)
obj_df.shape

(1386, 12)

In [37]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X= sc_X.fit_transform(num_df.values)
#scaled df numeric
num_df=pd.DataFrame(X,columns=num_df.columns.tolist())

In [38]:
df= pd.concat([num_df,obj_df,label_df],axis=1)

In [40]:
#drop highly correlated features
#Create correlation matrix
corr_matrix = df.corr().abs()

In [ ]:
# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
# Find index of feature columns with correlation greater than 0.9
to_drop = [column for column in upper.columns if any(upper[column] > 0.9)]
to_drop

In [ ]:
# Drop features 
df.drop(to_drop, axis=1, inplace=True)
df.head()

In [43]:
df.to_csv('f.csv',encoding='utf-8', index=False)

In [44]:
df.shape

(1386, 12)

In [45]:
from sklearn.model_selection import train_test_split
X_df = df.iloc[:, :-1]
y_df = df.iloc[:, 11]

In [46]:
X_train_df, X_test_df, y_train_df, y_test_df = train_test_split(X_df, y_df, test_size = 0.2)

ANN

In [47]:
classifier2 = Sequential()
classifier2.add(Dense(output_dim = 6, init = 'uniform', activation = 'relu', input_dim = 11))
classifier2.add(Dense(output_dim = 6, init = 'uniform', activation = 'relu'))
#classifier2.add(Dense(output_dim = 31, init = 'uniform', activation = 'relu'))
classifier2.add(Dense(output_dim = 1, init = 'uniform', activation = 'sigmoid'))
classifier2.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [48]:
classifier2.fit(X_train_df, y_train_df, batch_size = 10, nb_epoch = 100)

Epoch 1/100
1108/1108 [==============================] - 1s 470us/step - loss: 0.6933 - acc: 0.4919
Epoch 2/100
1108/1108 [==============================] - 0s 334us/step - loss: 0.6926 - acc: 0.5217
Epoch 3/100
1108/1108 [==============================] - 0s 253us/step - loss: 0.6889 - acc: 0.5388
Epoch 4/100
1108/1108 [==============================] - 0s 157us/step - loss: 0.6815 - acc: 0.5505
Epoch 5/100
1108/1108 [==============================] - 0s 186us/step - loss: 0.6730 - acc: 0.5578
Epoch 6/100
1108/1108 [==============================] - 0s 156us/step - loss: 0.6670 - acc: 0.5578 0s - loss: 0.6667 - acc: 0.558
Epoch 7/100
1108/1108 [==============================] - 0s 160us/step - loss: 0.6630 - acc: 0.5578
Epoch 8/100
1108/1108 [==============================] - 0s 167us/step - loss: 0.6603 - acc: 0.5578
Epoch 9/100
1108/1108 [==============================] - 0s 159us/step - loss: 0.6585 - acc: 0.5578
Epoch 10/100
1108/1108 [==============================] - 0s 164us/st

Epoch 80/100
1108/1108 [==============================] - 0s 175us/step - loss: 0.6369 - acc: 0.5785
Epoch 81/100
1108/1108 [==============================] - 0s 334us/step - loss: 0.6367 - acc: 0.5695
Epoch 82/100
1108/1108 [==============================] - 0s 222us/step - loss: 0.6364 - acc: 0.5821
Epoch 83/100
1108/1108 [==============================] - 0s 305us/step - loss: 0.6364 - acc: 0.5812
Epoch 84/100
1108/1108 [==============================] - 0s 188us/step - loss: 0.6367 - acc: 0.5812
Epoch 85/100
1108/1108 [==============================] - 0s 193us/step - loss: 0.6365 - acc: 0.5650
Epoch 86/100
1108/1108 [==============================] - 0s 179us/step - loss: 0.6362 - acc: 0.5785
Epoch 87/100
1108/1108 [==============================] - 0s 221us/step - loss: 0.6360 - acc: 0.5812
Epoch 88/100
1108/1108 [==============================] - 0s 204us/step - loss: 0.6358 - acc: 0.5803
Epoch 89/100
1108/1108 [==============================] - 0s 134us/step - loss: 0.6360 - ac

In [119]:
classifier.score(X_train_df,y_train_df)

0.5514440433212996

In [107]:
#train prediction
y_pred_train=classifier2.predict(X_train_df)
y_pred_train =np.where(y_pred_train > 0.5 ,1, 0)
cm_train= confusion_matrix(y_train_df, y_pred_train)
print(cm_train)

[[456  91]
 [377 184]]


In [108]:
total_train_records=456+184+91+377
print(total_train_records)
accuracy_train=((456+184)/total_train_records)*100
accuracy_train

1108


57.761732851985556

In [79]:
#test prediction
y_pred = classifier2.predict(X_test_df)
y_pred

array([[1.5656589e-08],
       [8.1643504e-01],
       [4.6702513e-01],
       [7.1493572e-01],
       [4.6726561e-01],
       [4.6766651e-01],
       [4.8965093e-01],
       [4.7071451e-01],
       [4.6862879e-01],
       [5.1648897e-01],
       [4.8965093e-01],
       [8.0035800e-01],
       [4.6806744e-01],
       [1.7805785e-08],
       [4.6758631e-01],
       [4.8965093e-01],
       [4.8965093e-01],
       [5.6926817e-01],
       [6.5766549e-01],
       [4.8965093e-01],
       [5.5841768e-01],
       [4.8965093e-01],
       [1.0510681e-02],
       [4.6702513e-01],
       [6.6454256e-01],
       [5.7258183e-01],
       [6.5596604e-01],
       [4.8965093e-01],
       [5.2364326e-01],
       [4.6886939e-01],
       [4.6734577e-01],
       [4.7039354e-01],
       [4.8965093e-01],
       [4.8965093e-01],
       [4.7095519e-01],
       [9.2406476e-01],
       [4.8965093e-01],
       [4.6758631e-01],
       [4.8965093e-01],
       [4.6846837e-01],
       [4.7047380e-01],
       [6.580291

In [81]:
y_pred =np.where(y_pred > 0.5 ,1, 0)
y_pred

array([[0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
    

In [83]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test_df, y_pred)
cm

array([[123,  23],
       [ 83,  49]], dtype=int64)

In [52]:
total_test_records=123+49+23+83
total_test_records

278

In [53]:
accuracy=(123+49)/total_test_records
accuracy*100

61.87050359712231

In [ ]:
df_test_ANN=X_test_df.copy()
df_test_ANN['Actual']=y_test_df.values
df_test_ANN['Predicted']=y_pred
df_test_ANN.head()

LogisticRegression

In [54]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(X_train_df, y_train_df)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [109]:
#train prediction
y_pred_train=classifier.predict(X_train_df)
cm_train= confusion_matrix(y_train_df, y_pred_train)
print(cm_train)

[[323 224]
 [273 288]]


In [110]:
total_train_records=323+288+224+273
print(total_train_records)
accuracy_train=((323+288)/total_train_records)*100
accuracy_train

1108


55.14440433212996

In [87]:
#test prediction
y_pred = classifier.predict(X_test_df)
y_pred

array([0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0,
       0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0,
       1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1,
       1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0,
       1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1,
       1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1,
       0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1,
       0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1,
       0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [88]:
cm = confusion_matrix(y_test_df, y_pred)
cm

array([[73, 73],
       [54, 78]], dtype=int64)

In [89]:
accuracy=(73+78)/total_test_records
accuracy*100

54.31654676258992

In [ ]:
df_test_LR=X_test_df.copy()
df_test_LR['Actual']=y_test_df.values
df_test_LR['Predicted']=y_pred
df_test_LR.head()

SVM

In [59]:
from sklearn.svm import SVC
classifier3 = SVC(kernel = 'linear', random_state = 0)
classifier3.fit(X_train_df, y_train_df)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=0, shrinking=True,
  tol=0.001, verbose=False)

In [111]:
#train pred
y_pred_train=classifier3.predict(X_train_df)
cm_train= confusion_matrix(y_train_df, y_pred_train)
print(cm_train)

[[276 271]
 [217 344]]


In [112]:
print(total_train_records)
accuracy_train=((276+344)/total_train_records)*100
accuracy_train

1108


55.95667870036101

In [92]:
#test pred
y_pred = classifier3.predict(X_test_df)
y_pred

array([0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0,
       0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0,
       0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0,
       1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1,
       1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0,
       1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1,
       0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1,
       0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1], dtype=int64)

In [93]:
cm = confusion_matrix(y_test_df, y_pred)
cm

array([[61, 85],
       [41, 91]], dtype=int64)

In [94]:
accuracy=(61+91)/total_test_records
accuracy*100

54.67625899280576

In [ ]:
df_test_SVM=X_test_df.copy()
df_test_SVM['Actual']=y_test_df.values
df_test_SVM['Predicted']=y_pred
df_test_SVM.head()

RandomForestClassifier

In [65]:
from sklearn.ensemble import RandomForestClassifier
classifier4 = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state=0)
classifier4.fit(X_train_df, y_train_df)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [113]:
#train pred
y_pred_train=classifier4.predict(X_train_df)
cm_train= confusion_matrix(y_train_df, y_pred_train)
print(cm_train)

[[517  30]
 [287 274]]


In [114]:
print(total_train_records)
accuracy_train=((517+274)/total_train_records)*100
accuracy_train

1108


71.38989169675091

In [97]:
#test pred
y_pred = classifier4.predict(X_test_df)
y_pred

array([0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0,
       0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1,
       0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1,
       1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0], dtype=int64)

In [98]:
cm = confusion_matrix(y_test_df, y_pred)
cm

array([[115,  31],
       [ 77,  55]], dtype=int64)

In [99]:
accuracy=(115+55)/total_test_records
accuracy*100

61.15107913669065

In [ ]:
df_test_RFC=X_test_df.copy()
df_test_RFC['Actual']=y_test_df.values
df_test_RFC['Predicted']=y_pred
df_test_RFC.head()

GaussianNB

In [73]:
from sklearn.naive_bayes import GaussianNB
classifier5 = GaussianNB()
classifier5.fit(X_train_df, y_train_df)

GaussianNB(priors=None)

In [115]:
#train pred
y_pred_train=classifier5.predict(X_train_df)
cm_train= confusion_matrix(y_train_df, y_pred_train)
print(cm_train)

[[ 40 507]
 [  0 561]]


In [116]:
print(total_train_records)
accuracy_train=((40+561)/total_train_records)*100
accuracy_train

1108


54.24187725631769

In [102]:
#test pred
y_pred = classifier5.predict(X_test_df)
y_pred

array([0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64)

In [103]:
cm = confusion_matrix(y_test_df, y_pred)
cm

array([[  9, 137],
       [  0, 132]], dtype=int64)

In [104]:
accuracy=(9+132)/total_test_records
accuracy*100

50.719424460431654

In [ ]:
df_test_NB=X_test_df.copy()
df_test_NB['Actual']=y_test_df.values
df_test_NB['Predicted']=y_pred
df_test_NB.head()